# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    
    # Wait to load the page.
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'html5lib')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "html5lib")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 「零接觸」、「宅經濟」　TAITRA談後疫時代數位商機及轉型爆發力

▲經濟部國際貿易局與外貿協會舉辦「後疫時代數位商機探索論壇」。（圖／TAITRA提供）
記者邱倢芯／台北報導
新冠病毒來襲，在「零接觸」及「宅經濟」的需求下，為讓業界了解後疫情時代數位經濟的趨勢，經濟部國際貿易局與外貿協會於今（27）日辦理「後疫時代數位商機探索論壇」，並同步於經貿透視Facebook粉絲團與外貿協會YouTube頻道線上直播，吸引線上線下共計超過500人次參加。
外貿協會董事長黃志芳表示，現今全球化程度已不可同日而語，全球經濟綿密交錯、互相連結，只要其中一個小小的環節出了問題，蝴蝶效應就會帶來意料之外的影響。
黃志芳更強調，在數位時代每一家企業都是科技公司，這波疫情讓數位轉型加快了5至10年，此時此刻不論是公部門或私人企業都體會到數位轉型的迫切性。數位化的程度將決定未來企業的生存與競爭力，外貿協會不僅展覽、產品發表會和買主拓銷洽談會等活動全面數位化，更開辦線上數位學院協助中小企業適應這波衝擊。
台灣區電機電子工業同業公會理事長暨前英業達董事長李詩欽提到，後疫時代製造業回穩，將朝在臺研發、在亞洲低成本國家量產、在銷售市場組裝發展，供應全球客戶。李詩欽強調5G是臺灣的未來、10年最大的商機，人工智慧、智慧製造、邊緣運算、雲端運算及大數據將影響企業的競爭力，圍繞在旁邊的就是數位轉型。
台灣微軟合作夥伴暨商務事業群總經理林劍福特別強調，數位轉型的重點不是數位、而是轉型。每個員工都要擁抱挑戰，改良產品為客戶解決難題與挑戰，用同理心了解客戶真正的需要，甚至與客戶建立夥伴關係，便能順利轉型產品。
在電商網購方面，台灣樂天市場財務長李志興表示，電商絕對是未來交易的重要平台，尤其電商可做到實體店面無法做到的大數據分析，了解客戶的需要而有精準的行銷。
TutorABC總經理黃嘉琦表示，大數據（Data）、互聯網（Network）與人工智慧（AI）將讓全世界成為一所學校，每個人都能有公平受教的機會，在數位平臺上獲得所需的知識與技能課程，更可以因材施教。訊連科技副總經理林昭宏則看好視訊會議軟體的發展，該公司更因應可能再發生的疫情，研發臉部口罩辨識系統，即

NCC表示，全球紀實為加強與各國影視產業交流，並承諾Asian Food Network亞洲美食頻道未來每年將採購或與臺灣影視團隊合製至少15小時本地相關節目，並於該頻道播出，有助於提升頻道內容豐富度，並帶動國內相關產業發展。
----------------------------------------------------------------------

[10] 捍衛「東森」品牌！王令麟：東森房屋將全面進行嚴選汰換　共創三贏

▲東森集團總裁王令麟表示，東森房屋致力提供會員更好的服務、深耕經營，持續貫徹「東森嚴選」的精神。（圖／記者周宸亘攝，以下同）
記者洪菱鞠／台北報導
東森集團秉持著顧客至上、以客為尊的品牌精神，今（27日）於東森房屋董事會中共同決議，將針對原有加盟店進行「東森嚴選」品質考核，從現有加盟店之經營、商譽與信用評等等多面向進行評估檢討，裁撤表現欠佳之加盟店、收回品牌，針對表現績優的加盟店，東森集團也將善用龐大的導客資源，進行線上線下整合、擴大各項業務推廣。東森集團總裁王令麟表示：「東森房屋不以加盟展店為經營目標，而是致力提供會員更好的服務、深耕經營，持續貫徹「東森嚴選」的精神。」
2020《天下雜誌》兩千大調查結果出爐，東森為虛擬零售業成長最快速的品牌，較前年成長20%，東森集團總裁王令麟更宣布2020年預計成長30%。根據《天下雜誌》調查，服務業2000大．東森自2017年的排名110名，至2019年晉升至排名86名，證明市場對於東森品牌的認同、信賴及喜好。
▲王令麟到東森房屋舉辦個人講座。
王令麟表示，東森基於愛屋及烏保護所有消費者，秉持一貫堅持，維持最佳的服務品質，將對旗下事業體東森房屋之加盟店將採取「東森嚴選」的品質辦理，同時對績優地區深耕經營以及對品牌有貢獻的單位，東森也將善用集團資源，將會員導客給各個加盟店，共創三贏：一贏強化東森的品牌、二贏提升加盟店的財源、三贏更是消費者權益的保障。
王令麟在企業經營上，始終堅持最優質的品質與服務，穩坐各產業領域第一名，包含東森購物、東森新聞雲、東森自然美與東森寵物雲等優質品牌。自2019年東森集團重拾東森房屋經營權以來，更全力投入改革，王令麟總裁認為必須打破過去品牌競爭、善用集團資源相互導客，透過實際行動幫助加盟店東賺大錢、拓展客源。
▲王令麟指出，將擦亮東森，成為更值得信賴、好品質的

▲受到肺炎疫情影響，4月景氣燈號再減1分，降為19分。（資料照／記者林敬旻攝）
記者邱倢芯／台北報導
國發會今（27）日公布4月景氣對策信號綜合判斷分數為19分，較上月減少1分，燈號續呈黃藍燈；景氣領先、同時指標持續下跌，國發會指出，這顯示出國內景氣仍受新冠肺炎（COVID-19）疫情干擾。
國發會指出，受到疫情擴散影響，各國採取必要措施以控制疫情，導致對於各國經濟產生負面影響，台灣因防疫措施超前部署，相關影響也較早產生。
4月份景氣受到批發、零售及餐飲營業額下滑影響，使分數下降，惟隨著國內疫情趨緩，零售及餐飲業等服務業表現，5月下旬逐步改善、消費回溫；製造業部份，雖然受到歐美市場疫情影響而需求趨緩，影響我國部份產業出口成績，然而因遠距應用產品需求激增，台灣電子零組件業生產指數持續成長，有助支持整體表現。
國發會進一步指出，未來因國際疫情後續發展未明，主要國家雖逐步解封重啟經濟，但消費、投資及生產活動短時間內尚未能回復疫前水準，加上美中貿易戰等影響，政府將持續密切關注對於台灣經濟影響，並妥善因應。為減緩衝擊，政府刻正籌備刺激國內消費、活絡商圈等振興措施；並加速推動投資台灣三大方案、提升公共建設執行率等，力拚經濟動能增加、就業市場穩定。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[20] 藍弋丰／三十年一覺出版夢　贏得十趴薄倖名（下）

 ▲ 早期書本能否暢銷，看出版社是否有砸錢行銷。（示意圖／記者李毓康攝）
前回提到「十趴之亂」根本上來說還是書市萎縮，大家的十趴都縮水，所以委屈了。另一方面，時代的改變，造就「網紅」出書。網紅相對於傳統作者，談判籌碼的提升，是另一個重要因素。
30年前缺發表管道　新人夢想是登《皇冠雜誌》
30年前不像今日人人可當網紅，發表空間僅限於實體出版管道，不論是報紙與雜誌專欄，還是出版書籍，新人要能擠上欄位，那是莫大的幸運，或是有高深的人脈才能做到。
很多老牌作家年輕時，曾經最大的夢想就是能登上《皇冠雜誌》，但現在的新一代年輕網紅作家，可能很多已經沒聽說過《皇冠雜誌》。
在當年，若純粹只有文筆好，由於缺乏發表管道，「十年寒窗無人問」的情況非常多，當作者面對出版社這個重要發表管道，都是戰戰兢兢，雖然當年

文／中央社記者張茗喧、吳欣紜台北27日電
指揮中心今天宣布6月1日起國產口罩將部分開放內外銷，指揮官陳時中強調，庫存很夠、可盡量買；經濟部長沈榮津表示，由於與實名制雙軌並行，每片口罩應會低於5元。
中央流行疫情指揮中心今天下午召開記者會，經濟部長沈榮津在會中表示，口罩產能從一開始188萬片，透過政府採購機器跟民間等陸續加入生產行列，口罩國家隊達日產能2000萬片，疫情穩定後，購買需求也在下降，已經完成階段性任務，預估每天需求約800萬片，因此6月1日將改採定額徵用。
沈榮津表示，自1月31日全面徵用國產口罩以來，已累計徵用超過13億片口罩，庫存也有2億到3億片，目前台灣民眾口罩需求已趨緩且穩定，此時正是將短期防疫任務，轉化為長期發展戰略物資產業的好時機。
沈榮津說，6月1日起，國軍支援口罩生產及相關激勵措施將逐步退場，國內生產的醫療口罩改為每日定額徵用800萬片，其餘產能開放內外銷，並取消台灣民眾每人每2個月寄送30片口罩給海外二親等親屬限制。
不過，未來國內口罩庫存若低於1億片，則提高每日定額徵用量至1200萬片以補充庫存，以確保有疫情時，有足夠數量的口罩可供應國內民眾。
▲口罩國家隊已經完成階段性任務。（圖／記者唐詠絮攝）
外界擔心口罩回歸自由市場價格遭哄抬或存量不夠，陳時中表示，由於14天9片實名制規定仍並行，若擔心大量購買，國內口罩庫存量很夠，大家可以盡量買。
市售口罩價格方面，沈榮津指出，口罩開放銷售後，以健保卡購買的14天9片實名制機制仍不變，其餘將恢復自由買賣和市場機制，考量實名制價格仍為每片5元，廠商、通路商必須考慮到實名制5元，定價時每片一定會低於5元，由民眾自己選擇。
沈榮津強調，由於一般醫用口罩及外科手術口罩仍屬「刑法」第251條行政院公告的生活必需用品，不得有囤積居奇哄抬物價行為；而根據「公平交易法」第15條，業者之間也不得有合意或共同調漲口罩價格等聯合行為。
口罩禁令解除、開放自由市場買賣消息一出，沈榮津說，昨天口罩業者提到，在訂單方面，有很多電話來詢問，貿易商甚至海外通路商都是多元詢價，訂購量有千萬片或者是以億為單位的大單都有，希望國內各廠商一起打國際盃。（編輯：陳清芳）
----------------------------------------------------------------------

[26] 全球走入低利率

▲ 台股。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
台股今（27）日早盤開高，最高一度上漲72點、來到11069點，創兩個半月新高，但盤中一度翻跌，甚至跌破「萬一」關口，不過隨後又反彈上揚，指數在11000點之上震盪，終場收在11014點、上漲17點或0.16%，成交量1562.36億元。
蘋概三王中，台積電（2330）以296.5元作收、上漲0.34%；鴻海（2317）收在75.3元、上漲0.6%；股王大立光（3008）收3880元、下跌1.15%。
權值股中漲跌互見，台塑四寶全數下跌，跌幅約在0.14%~0.67%之間；金控雙雄雙雙收紅。
類股指數同樣也漲跌互見，多呈小漲小跌，其中電器類股漲勢較大、漲幅4.24%，電機類股也有1.88%漲幅；跌勢較重的類股則是生醫類股，下跌2.96%。
兆豐投顧董事長李秀利指出，今天大家都在關注生技股中的獲利回吐以及股價籌碼的變數，而台股盤中震盪壓回到平盤下，一部分原因也是受生技類股拖累，另一部分是因為萬一點以上追價買盤會相對謹慎，唯有OTC指數中的中小型股較不受影響，整個OTC周線連九紅，但今天攻高力道和大盤差不多，有一點量縮的情況。
李秀利表示，隨著疫情趨緩，生技防疫概念股休息之後，市場部分資金轉向觀光類股，另外電器電纜及電子零組件表現也較為強勢，其中電器電纜類股主要是受到集團股裡的華新所拉抬，扣除華新以外，其他電器電纜股表現就較為持平。
對於投資人操作建議，李秀利表示，由於中國大陸兩會期間，政策出台利多產業主要為5G、「兩新一重」，所以資金依然會在電子和傳產類股間輪動；另外，受到各國鎖國政策逐漸鬆綁，資金也逐漸留向觀光、航運類股，投資人可留意佈局。
----------------------------------------------------------------------

[35] 台彩端午加碼7億創史上新高！　大樂透加開「百組百萬」

▲台彩端午加碼活動6月1日開跑，總加碼金達新台幣7億元，創史上新高。（圖／台彩提供）
記者紀佳妘／台北報導
台灣彩券公司今（27）日宣布端午節加碼活動，祭出史上端午加碼總獎金跟加碼遊戲種類最多的重量級加碼，威力彩機動加碼2億元，大樂透增開百組百萬獎，今彩539頭獎也提高至1,000萬元，總加碼金高達7億元，創下史上新高。
台彩指出，端午加碼活動自6月1日起跑，首先登

▲華碩董事長施崇棠。（圖／國研院提供）
記者姚惠茹／台北報導
華碩（2357）今（27）日宣布，每年固定於畢業季舉辦的校園徵才活動，今年特別配合中央落實防疫，改採線上說明會形式，從5月29日起至7月中接力登場，並邀請多位華碩中、高階主管現身說法，讓即將投入職場的新鮮人快速瞭解華碩內部企業文化與職務所需具備的特質。
華碩表示，這次招募的領域橫跨「AI人工智慧」、「軟韌體研發」、「硬體研發」、「產品經理」、「業務行銷」與「客戶服務」等，而且觀看線上說明會並掃描影片中QR Code登錄履歷，就有機會獲得華碩新一代「ROG Phone II」電競手機。
此外，華碩指出，自2014年首次對外徵招對科技懷抱熱忱，以及兼具創新精神國際型人才的「Global Talent Program (GTP)」邁入第七屆，今年不僅擴大招募，日前更特別請來擁有180萬總訂閱數的超人氣YouTuber—「Joeman」到華碩企業總部拍攝一日員工體驗影片，引發網友熱烈討論。
華碩強調，凡工作經驗未滿兩年，並對於產品經理、行銷、業務及客服等相關領域有興趣的求職者，敬請把握難得機會至華碩GTP官方網站或華碩人才招募網瞭解職缺、填寫履歷，錄取者將有機會於華碩企業總部接受3個月課程訓練，以及4~8個月的專業在職訓練，再依照適性分派至總部或全球業務據點。
▲華碩校園線上徵才開跑。（圖／華碩提供）
----------------------------------------------------------------------

[46] 影／買咖啡免「出國」！小港第一家星巴克街邊店將開　百坪空間估月租僅5位數

記者陳建宇／高雄報導
星巴克積極展店，特別是具備「得來速」車道的路邊店愈開愈多，高雄小港近期內也將開出第一家路邊店，位置就在南小港最熱鬧的漢民商圈。房仲業者指出，星巴克的進駐，可望吸引其他大型連鎖商家也進駐小港，對於提升商圈價值幫助大。
▲小港區第1家星巴克路邊店近期將開幕，可望帶動商圈價值。（圖／記者陳建宇攝）
▲漢民商圈是南小港最熱鬧的區段，店家非常多。（圖／記者陳建宇攝）
該店的位置在宏平路、龍華街口，屬於南小港最熱鬧的漢民商圈，住商不動產小港宏平加盟店副店長丁晉偉表示，漢民商圈主要範圍包括漢民路、金府路、康莊路及宏平路等路段，店家多且密集，食衣住行育樂等生活機能都可以滿足，商圈內還

▲誠品生活今（27）日召開股東會，董事長吳旻潔表示將持續強化全通路工程的建構。（圖／記者湯興漢攝）
記者林淑慧／台北報導
誠品生活今（27）日召開股東會，董事長吳旻潔表示，受到新冠肺炎疫情衝擊，上半年業績「三窮四絕」，隨著亞洲各國逐步解封，業績逐月回升，今年將奠基在290萬記名會員之上，朝向「海內外文化觀光品牌」、「台灣文創全通路平台」二大方向前進，持續強化全通路工程的建構。
針對疫情影響，吳旻潔指出，上半年歷經最慘淡的四月，疫情在上半年的影響可說是「三窮四絕」，誠品於二月重新訂定目標，把握時間調整體質，後續將果斷結束不具獲利與開源性的事業體，並積極爭取政府補助、與房東洽談降租，將營運成本降至最低，以時間換取空間。
誠品經重新審視長期合約，吳旻潔指出，預計於下半年再關一至二家到期的門市，主要為百貨公司高樓層、小坪數的門市，以及位於老社區的站點將關舊開新，目前已在積極選址。
▲誠品生活股東常會今(27)日召開，會中董事長吳旻潔指出由於疫情影響，誠品第一季成為史上經營最慘淡的一季。（圖／記者湯興漢攝）
「不要害怕關店，開錯店不是好事，開對店才是重點。」吳旻潔說，關閉不具未來性的店別，建立可複製量化的店型模組，為今明兩年強化誠品營運的重點，並向股東喊話，看到誠品關店不需擔心，是誠品「先蹲後跳」的基礎。
另外，誠品生活也將插旗馬來西亞吉隆坡，首店規模估達上千坪，原先計畫在明年開出東南亞首店，吳旻潔指出，但受到疫情影響，時間點可能還有變化；此外，誠品加速發展全通路考慮增資，也打算在台灣開社區小店。
吳旻潔表示，全通路是決定勝負的關鍵，誠品生活將勢必得由實體通路走向全通路，今年將朝向「創新既有營運模式」與「研發個人化服務」方向持續投資，創新既有營運模式包含創新店型、品牌投資與代理、品牌授權與顧問經營，研發個人化服務則著重全通路基礎建設、社群經營與數位服務。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[56] 新台幣開盤貶0.2分　以29.990元開出

▲新台幣兌美元今天開盤貶值0.2分。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（27）日開盤貶值0.2分，以29.990元兌1美元開出。
新台幣匯率昨（

▲口罩進口關稅明日起恢復原稅率。（圖／取自免費圖庫Pixabay）
記者吳靜君／台北報導
為了因應新冠肺炎（COVID-19）疫情，關務署自2月27日起，調降取得衛福部核發證明文件藥用酒精原料與口罩進口關稅3個月，至今（26）日到期。關務署宣布，明（27）日起酒精原料進口關稅減半，維持關稅10%繼續延期3個月到8月26日；口罩則恢復原稅率7.5%。
中央流行疫情指揮中心宣布口罩將於6月1日解禁，指揮中心未來將每天徵用800萬片，其餘開放自由販售；口罩出口也將解除禁令，若經濟部全面解禁出口，民眾寄送口罩、旅客攜帶口罩出境，也將可不受限制。
原本關務署向行政院建議，因為民眾買口罩還實名制，所以口罩進口原料免關稅、酒精原料進口關稅減半再延長3個月，到8月26日。
但是隨著台灣口罩產量增加，每日大約可生產1900萬片到2000萬片，而且疫情指揮中心也宣布6月1日起口罩解禁、出口也解禁等陸續鬆綁措施，加上提升國內廠商口罩競爭力，因此決議口罩恢復原稅率7.5%。至於，藥用酒精的部分因為國外的酒精原物料仍然非常的高昂，所以酒精原料可以再延長3個月，稅率仍為10%。
口罩與酒精原料的進口關稅稅率彈性調整，是依據關稅法第71條規定，為應付國內或國際經濟的特殊情況，並調節物資供應及產業合理經營，對進口貨物應徵的關稅或適用的關稅配合得在海關進口稅則規定的稅率或數量50%以內予以增減。但大宗物資價格大幅波動時，得以100%內予以增減。實際增減之幅度及開始與停止日期，由財政部會商有關機關擬訂，報請行政院核定。
----------------------------------------------------------------------

[69] 取代矽基！手機晶片運用碳基技術...加快5倍速、省電達3倍

記者邱茀濱／綜合報導
超越了傳統技術！中國在碳基半導體製備材料上取得關鍵性突破，並從實驗室研究向工業化運用推進了一大步。據悉，碳基技術應用在智慧手機晶片上，至少提升5倍的速度，甚至節省3倍的電量。
據陸媒報導，北京北京元芯碳基集成電路研究院院士彭練矛指出，「採用矽以外的材料做集成電路，包括鍺、砷化鉀、石墨烯和碳，一直是國外半導體前沿的技術。而碳基半導體則具有成本更低、功耗更小、效率更高的優勢，更適合在不同領域的應用而成為更好的半導體材料選項。我們的碳基半導體研究是代表世界領

▲財政部。（圖／資料照）
記者吳靜君／台北報導
振興券要來了！政府規劃要發放振興券，民眾消費1000元，就可以換得3000元的振興券，並且也以多元方式發放，為了提升「台灣Pay」的使用率、鼓勵民眾使用振興券綁定台灣Pay，財政部預計下周四（6月4日）邀公股銀行討論民眾振興券綁台灣pay的優惠，讓民眾「利上加利」。
隨著新冠肺炎（COVID-19）國內疫情逐漸趨緩，進入振興經濟、促進消費的階段，目前政府所規劃振興券方案，民眾花1000元，可以換3000元振興券，等於打了三三折，也大幅放寬消費的項目與使用地點，以期望可以發揮刺激消費的效果。經濟部目前已經定調，振興券將以多元方式發放，包含實體券、綁定行動支付、電子票證以及信用卡、電子票證等方式，讓民眾可以方便領取以及方便使用。
為了因應「振興券」發放和鼓勵民眾使用「電子支付」來綁定與消費，財政部將找八大公股銀行開會，推廣振興券綁定「台灣Pay」。根據了解，公股銀行會先討論方案，並且提供財政部共同討論，預計6月4日之後有一些定論。
目前公股銀行規劃大約三方向：一為直接回饋給消費者，像是編列預算讓民眾綁定台灣Pay，花3000元的消費就給予回饋，等於有政府的振興券之外，還可以另外賺200元到300元；另外，公股銀行也考慮給合作店家一些優惠，鼓勵店家辦理促銷活動，一方面可以促進更多店家與台灣Pay合作，另一方面也鼓勵店家吸引民眾上門消費。最後，公股銀行也考慮辦抽獎活動，像是抽iPhone等大獎都在範圍之內。
信用卡的部分各銀行也會另外提出優惠，鼓勵民眾使用振興券不用實體券，可以「利上加利」。根據銀行預估，目前這四種發放方式，民眾領取實體券或使用信用卡的方式會超過七成、八成；其他才會綁定行動支付以及電子票證。
由於振興券即將在7月上路，在時程規劃上較為緊張，所以公股銀預計在6月4日開會討論後會有具體的方向；而該項會議也將由財政部次長阮清華主持。
～好康來了！過期票券換東森幣
----------------------------------------------------------------------

[79] 檢察官首例轉任金管會副主委　許永欽上任後借鏡經驗提4大協助

▲金管會副主委許永欽。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
史上首位從檢察官轉任金管會副主委的許永欽今（26）日表示，過去彙整許多企業不法

▲哈佛大學政治學教授艾利森（Graham Allison）。（圖／達志影像／美聯社）
記者王曉敏／綜合報導
隨著貿易戰及5G角力加劇，中美關係逐年緊張，哈佛大學政治學教授艾利森（Graham Allison）今（26）日接受外媒採訪時表示，中美關係惡化，最終的局面將是「雙輸」。
艾利森今日在CNBC節目《 Squawk Box Asia》上表示，美國總統川普與中國國家主席習近平正努力控制其國內的疫情，也持續就病毒源頭問題相互指責。「我認為這將使情況全面惡化，我希望這不會造成永久性的損害。」他指出，這兩個經濟巨人間的競爭可能導致第一階段中美貿易協議的破裂。
事實上，除了艾利森外，其他專家也警告，北京當局將難以履行其在第一階段貿易協議中祭出的承諾，即中國大陸將在2017年的基礎上，於2021年前額外購買2000億美元的美國商品即服務。戰略與國際研究中心（CSIS）智庫本月中旬即預期，受到新冠肺炎疫情影響，這一承諾將難以達成。
日前有專家指出，中美間的緊張局勢可能導致「新冷戰」。曾於川普政府擔任貿易顧問、現為律師事務所艾金甘普合夥人的威廉斯（Clete Willems）本月初就說道：「我們必須誠實地面對現實，這是新冷戰的開始，如果我們不小心，事情恐變得更糟。」
不過，布魯金斯學會（Brookings Institution）研究員李成表示，即使兩國關係的惡化速度較預期快得多，但雙方都未對此作好準備，「我認為戰爭將是毀滅性的，不會有贏家。」他強調，「這場戰爭應該且可以並必須被預防。」
----------------------------------------------------------------------

[90] 還有降息的空間　永豐金：市場認為央行Q2可望降息1碼

▲永豐金控表示，目前市場認為央行還有降息的一點點空間。（圖／資料照）
記者吳靜君／台北報導
央行將在6月中旬召開理監事會議，決定利率，外界關心利率走向，以及影響銀行的獲利情形，永豐金今（26）日召開法說會，永豐金總經理朱士廷表示，目前整個局勢皆認為還有一點點降息的空間，尤其美國已經降到幾乎「零利率」，現在市場上普遍認為，央行第二季會降息約一碼（0.25%）。
朱士廷先表達對今年的看法「謹慎樂觀」，認為今年第二季經濟成長率會看到明顯的低點，但是下半年國際疫情趨緩，景氣會回升，第三季、第四季